In [1]:
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

2.4.0
2.4.3


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import time

pd.set_option('display.max_columns', 70)

start = time.time()

In [4]:
def scale_data(data):
    list_of_features = []
    list_of_outputs = []
    for column in data.columns:
        list_of_features.append(data[column])
        if column in ['Open', 'Close', 'Highest', 'Lowest', 'Volume', 'Otwarcie', 'Najwyzszy', 'Najnizszy', 'Zamkniecie', 'Wolumen']:
            list_of_outputs.append(data[column])
        
    dataset = np.transpose(list_of_features)
    output_dataset = np.transpose(list_of_outputs)
    X_scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = X_scaler.fit_transform(dataset)
    Y_scaler = MinMaxScaler(feature_range=(0,1))
    Y_scaler.fit_transform(output_dataset)

    return scaled_data, X_scaler, Y_scaler

In [5]:
def prepare_input_and_output(data, number_of_sessions=60):
    # number_of_sessions - number of considered previous sessions as an input
    X = []
    Y = []
    for i in range(number_of_sessions, data.shape[0]):
        X.append(data[i-number_of_sessions:i, :])
        Y.append(data[i, :5])
    return X, Y

In [6]:
fundamental_data = pd.read_csv('Testing_data/WiG20_full_data.csv')

fundamental_data.head()

,Spółka,Data,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen,Kwartały,Wartość księgowa,Cena/WK,Zysk,Cena/Zysk,Przychód,Cena/Przychód,Zysk operacyjny,Cena/Zysk operacyjny,Wartość księgowa Grahama,Cena/Wartość księgowa Grahama,Wartość przedsiębiorstwa,Wartość przedsiębiorstwa/Przychody,Wartość przedsiębiorstwa/EBIT,Wartość przedsiębiorstwa/EBITDA,ROE,ROA,Marża zysku operacyjnego,Marża zysku netto,Marża zysku ze sprzedaży,Marża zysku brutto,Marża zysku brutto ze sprzedaży,Rentowność operacyjna aktywów,Udział zysku netto w przepływach operacyjnych,Wskaźnik źródeł finansowania inwestycji,Zadłużenie ogólne,Zadłużenie kapitału własnego,Zadłużenie długoterminowe,Zadłużenie środków trwałych,Pokrycie aktywów trwałych kapitałami stałymi,Trwałość struktury finansowania,Zastosowanie kapitału obcego,Wskaźnik ogólnej sytuacji finansowej,Zadłużenie netto,Zadłużenie netto / EBITDA,Zadłużenie finansowe netto,Zadłużenie finansowe netto / EBITDA,I stopień pokrycia,II stopień pokrycia,Płynność gotówkowa,Płynność szybka,Płynność bieżąca,Płynność podwyższona,Pokrycie zobowiązań należnościami,Udział kapitału pracującego w aktywach,Pokrycie kosztów kapitałem obrotowym,Rotacja należności,Cykl należności,Cykl zobowiązań,Rotacja zapasów,Cykl zapasów,Rotacja majątku obrotowego,Rotacja majątku trwałego,Rotacja majątku ogółem,Cykl operacyjny,Cykl konwersji gotówki
0,ACP,2005-02-28,16.358,16.854,16.358,16.526,492612.0,2004/Q4,6.55,2.50,0.02,817.90,23.01,0.71,1.03,15.88,-4.77,-3.43,33.99,1.48,21.97,33.1,0.32,0.08,4.46,0.09,5.43,-1.11,19.58,4.06,-1.68,108.82,0.73,2.84,0.63,0.34,0.93,0.42,1.34,0.77,292874000.0,9.04,127506000.0,3.94,0.57,0.93,0.16,0.89,0.96,0.32,0.55,-0.02,-0.04,2.89,126.0,229.0,19.74,18.0,1.66,2.0,0.91,144.0,-85.0
1,ACP,2005-03-01,16.741,16.741,15.979,16.308,293232.0,2004/Q4,6.55,2.56,0.02,837.05,23.01,0.73,1.03,16.25,-4.77,-3.51,33.99,1.48,21.97,33.1,0.32,0.08,4.46,0.09,5.43,-1.11,19.58,4.06,-1.68,108.82,0.73,2.84,0.63,0.34,0.93,0.42,1.34,0.77,292874000.0,9.04,127506000.0,3.94,0.57,0.93,0.16,0.89,0.96,0.32,0.55,-0.02,-0.04,2.89,126.0,229.0,19.74,18.0,1.66,2.0,0.91,144.0,-85.0
2,ACP,2005-03-02,16.308,16.414,15.594,15.817,163743.0,2004/Q4,6.55,2.49,0.02,815.40,23.01,0.71,1.03,15.83,-4.77,-3.42,33.99,1.48,21.97,33.1,0.32,0.08,4.46,0.09,5.43,-1.11,19.58,4.06,-1.68,108.82,0.73,2.84,0.63,0.34,0.93,0.42,1.34,0.77,292874000.0,9.04,127506000.0,3.94,0.57,0.93,0.16,0.89,0.96,0.32,0.55,-0.02,-0.04,2.89,126.0,229.0,19.74,18.0,1.66,2.0,0.91,144.0,-85.0
3,ACP,2005-03-03,15.817,16.035,15.650,15.650,116137.0,2004/Q4,6.55,2.41,0.02,790.85,23.01,0.69,1.03,15.36,-4.77,-3.32,33.99,1.48,21.97,33.1,0.32,0.08,4.46,0.09,5.43,-1.11,19.58,4.06,-1.68,108.82,0.73,2.84,0.63,0.34,0.93,0.42,1.34,0.77,292874000.0,9.04,127506000.0,3.94,0.57,0.93,0.16,0.89,0.96,0.32,0.55,-0.02,-0.04,2.89,126.0,229.0,19.74,18.0,1.66,2.0,0.91,144.0,-85.0
4,ACP,2005-03-04,15.707,15.923,15.650,15.923,48766.0,2004/Q4,6.55,2.40,0.02,785.35,23.01,0.68,1.03,15.25,-4.77,-3.29,33.99,1.48,21.97,33.1,0.32,0.08,4.46,0.09,5.43,-1.11,19.58,4.06,-1.68,108.82,0.73,2.84,0.63,0.34,0.93,0.42,1.34,0.77,292874000.0,9.04,127506000.0,3.94,0.57,0.93,0.16,0.89,0.96,0.32,0.55,-0.02,-0.04,2.89,126.0,229.0,19.74,18.0,1.66,2.0,0.91,144.0,-85.0


In [7]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [8]:
# Firstly we learn based on prices and volume only as a reference point for further predictions
X_all_ref = []
Y_all_ref = []
X_scalers_ref = []
Y_scalers_ref = []
results = []

reference_data = fundamental_data[['Spółka', 'Otwarcie', 'Najwyzszy', 'Najnizszy', 'Zamkniecie', 'Wolumen']].dropna()

for stock in list(reference_data['Spółka'].unique()):
    scaled_stock, X_scaler, Y_scaler = scale_data(reference_data.loc[reference_data['Spółka'] == stock].drop(columns=['Spółka']))
    X_scalers_ref.append(X_scaler)
    Y_scalers_ref.append(Y_scaler)
    X_single, Y_single = prepare_input_and_output(scaled_stock)
    X_all_ref = X_all_ref + X_single
    Y_all_ref = Y_all_ref + Y_single
X_all_ref, Y_all_ref = np.array(X_all_ref), np.array(Y_all_ref)
X_train_ref, X_valid_ref, Y_train_ref, Y_valid_ref = train_test_split(X_all_ref, Y_all_ref, test_size=0.2, shuffle=False)

print(X_train_ref.shape)
print(X_valid_ref.shape)
print(Y_train_ref.shape)
print(Y_valid_ref.shape)

loss_results_ref = []
val_loss_results_ref = []
for i in range(3):
    model = Sequential()
    model.add(LSTM(32, input_shape=[X_train_ref.shape[1], X_train_ref.shape[2]]))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(Y_train_ref.shape[1]))
    model.compile(loss='mse', optimizer='adam')
    history = model.fit(X_train_ref, Y_train_ref, epochs=7, validation_data=(X_valid_ref, Y_valid_ref))
    loss_results_ref.append(history.history['loss'][-1])
    val_loss_results_ref.append(history.history['val_loss'][-1])
results.append({'indicator': 'reference', 'loss': np.mean(loss_results_ref), 'val_loss': np.mean(val_loss_results_ref)})
results = pd.DataFrame(results)

(48433, 60, 5)
(12109, 60, 5)
(48433, 5)
(12109, 5)
Epoch 1/7
1514/1514 [==============================] - 10s 5ms/step - loss: 0.0279 - val_loss: 0.0011
Epoch 2/7
1514/1514 [==============================] - 6s 4ms/step - loss: 6.3059e-04 - val_loss: 9.6770e-04
Epoch 3/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.8806e-04 - val_loss: 8.8839e-04
Epoch 4/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.6128e-04 - val_loss: 8.0225e-04
Epoch 5/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.0311e-04 - val_loss: 8.6278e-04
Epoch 6/7
1514/1514 [==============================] - 6s 4ms/step - loss: 4.8032e-04 - val_loss: 7.9726e-04
Epoch 7/7
1514/1514 [==============================] - 6s 4ms/step - loss: 5.3117e-04 - val_loss: 8.4085e-04
Epoch 1/7
1514/1514 [==============================] - 8s 4ms/step - loss: 0.0323 - val_loss: 0.0011
Epoch 2/7
1514/1514 [==============================] - 6s 4ms/step - loss: 6.0436e-04 - val

In [ ]:

# iterate through columns as only one at a time is added to the reference data
for column in fundamental_data.columns[8:]:
    X_all = []
    Y_all = []
    X_scalers = []
    Y_scalers = []
#     print(column)
    data = fundamental_data[['Spółka', 'Otwarcie', 'Najwyzszy', 'Najnizszy', 'Zamkniecie', 'Wolumen', column]].dropna()
    
    # preparation of input X and output Y sets needs to be done on stocks seperately
    for stock in list(data['Spółka'].unique()):
        scaled_stock, X_scaler, Y_scaler = scale_data(data.loc[data['Spółka'] == stock].drop(columns=['Spółka']))
        X_scalers.append(X_scaler)
        Y_scalers.append(Y_scaler)
    
        X_single, Y_single = prepare_input_and_output(scaled_stock)
        X_all = X_all + X_single
        Y_all = Y_all + Y_single
    X_all, Y_all = np.array(X_all), np.array(Y_all)
    X_train, X_valid, Y_train, Y_valid = train_test_split(X_all, Y_all, test_size=0.2, shuffle=False)
#     print('Shape of training input matrix: ' + str(X_train.shape))
#     print('Shape of training output matrix: ' + str(Y_train.shape))
#     print('Shape of validation input matrix: ' + str(X_valid.shape))
#     print('Shape of validation output matrix: ' + str(Y_valid.shape))
    
    # neural network
    loss_results = []
    val_loss_results = []
    for i in range(3):
        model = Sequential()
        model.add(LSTM(32, input_shape=[X_train.shape[1], X_train.shape[2]]))
        model.add(Dense(10, activation='relu'))
        model.add(Dense(Y_train.shape[1]))
        model.compile(loss='mse', optimizer='adam')
        history = model.fit(X_train, Y_train, epochs=7, validation_data=(X_valid, Y_valid), verbose=0)
        loss_results.append(history.history['loss'][-1])
        val_loss_results.append(history.history['val_loss'][-1])
    results = results.append({'indicator': column, 'loss': np.mean(loss_results), 'val_loss': np.mean(val_loss_results)}, ignore_index=True)
    print(results.iloc[-1])

    

Wartość księgowa
indicator    Wartość księgowa
loss                   0.0005
val_loss             0.000853
Name: 2, dtype: object
Cena/WK
indicator     Cena/WK
loss         0.000499
val_loss     0.000847
Name: 3, dtype: object
Zysk
indicator        Zysk
loss         0.000497
val_loss     0.000831
Name: 4, dtype: object
Cena/Zysk
indicator    Cena/Zysk
loss          0.000498
val_loss      0.000877
Name: 5, dtype: object
Przychód
indicator    Przychód
loss         0.000505
val_loss     0.000854
Name: 6, dtype: object
Cena/Przychód
indicator    Cena/Przychód
loss              0.000497
val_loss          0.000832
Name: 7, dtype: object
Zysk operacyjny
indicator    Zysk operacyjny
loss                0.000492
val_loss             0.00085
Name: 8, dtype: object
Cena/Zysk operacyjny


In [ ]:
results

In [ ]:
end = time.time()
print('Time of executing the script: ' + str(round((end - start)/3600, 2)) + 'h')